Installing and Impoting Libraries like Numpy,Pnadas,Technical Analysis &
Reading The Dataset

In [ ]:
!pip3 install pandas
!pip3 install numpy
!pip3 install ta
!pip3 install pandas_ta

^C


In [1]:
import pandas as pd
import numpy as np
import ta
from ta.volatility import AverageTrueRange
from ta.trend import MACD, AroonIndicator
from ta.momentum import RSIIndicator
dat=pd.read_csv("bitcoin_2017_to_2023.csv")
data1=dat


##### Defining various functions and parameters such that-:
- Hawkes_Process
- Heiken Ashi Candle
- Fibbonaci Retracement
- Common Technical Indicators
- Long & Short weighted signals

In [2]:
"""
Hawkes Process (Discrete Form)

This function implements a discrete Hawkes-style self-exciting process, where past
events increase the current intensity but decay exponentially over time.

Input series: x_t
Decay rate: kappa > 0
Decay factor: alpha = exp(-kappa)

Recursive update:
H_t = alpha * H_(t-1) + x_t

Final intensity scaling:
lambda_t = kappa * H_t

Recent events contribute more than older ones, making this useful for modeling
event clustering, volatility, or order flow in financial time series.
"""


def hawkes_process(data: pd.Series, kappa: float):
    assert(kappa > 0.0)
    alpha = np.exp(-kappa)
    arr = data.to_numpy()
    output = np.zeros(len(data))
    output[:] = np.nan
    for i in range(1, len(data)):
        if np.isnan(output[i - 1]):
            output[i] = arr[i]
        else:
            output[i] = output[i - 1] * alpha + arr[i]
    return pd.Series(output, index=data.index) * kappa

In [3]:
"""
Heikin-Ashi Candlestick Calculation

This function computes Heikin-Ashi (HA) candles, which smooth price data
to reduce noise and better visualize trends.

Formulas:
HA_Close = (open + high + low + close) / 4
HA_Open  = (previous open + previous close) / 2
HA_High  = max(HA_Open, HA_Close, high)
HA_Low   = min(HA_Open, HA_Close, low)

Heikin-Ashi candles do not represent exact market prices. Instead, they
provide a trend-smoothed view where strong trends appear as consecutive
candles of the same color with smaller wicks.

Used for:
- Trend identification
- Noise reduction
- Momentum-based strategies
"""


def calculate_heikin_ashi(df):
    ha_df = df.copy()
    ha_df['HA_Close'] = (df['open'] + df['high'] + df['low'] + df['close']) / 4
    ha_df['HA_Open'] = (df['open'].shift(1) + df['close'].shift(1)) / 2
    ha_df['HA_High'] = ha_df[['HA_Open', 'HA_Close', 'high']].max(axis=1)
    ha_df['HA_Low'] = ha_df[['HA_Open', 'HA_Close', 'low']].min(axis=1)
    return ha_df[['HA_Open', 'HA_High', 'HA_Low', 'HA_Close']]

In [4]:
"""
Fibonacci Retracement Level Calculation

This function calculates key Fibonacci retracement levels based on a selected
price range. The range can be the full swing (entire dataset) or a recent
window (last 20 periods).

High and Low Selection:
- Swing mode: highest high and lowest low of the full data
- Recent mode: highest high and lowest low of the last 20 periods

Formulas:
Range = high - low
100%  = high
61.8% = high - Range * 0.618
50%   = high - Range * 0.5
38.2% = high - Range * 0.382
0%    = low

Fibonacci levels are used as potential support and resistance zones where
price may retrace before continuing the trend.
"""

def calculate_fibonacci(df, period='swing'):
    high = df['high'].max() if period == 'swing' else df['high'].iloc[-20:].max()
    low = df['low'].min() if period == 'swing' else df['low'].iloc[-20:].min()
    levels = {
        "100%": high,
        "61.8%": high - (high - low) * 0.618,
        "50%": high - (high - low) * 0.5,
        "38.2%": high - (high - low) * 0.382,
        "0%": low
    }
    return levels

In [8]:
"""
Technical Indicator Calculation

This function computes a set of momentum, trend, and volume-based technical
indicators commonly used in trading strategies.

Indicators included:
- RSI (14): Measures momentum by comparing recent gains to losses
- FastVolumeSMA (10): Short-term average of trading volume
- SlowVolumeSMA (50): Long-term average of trading volume
- FastEMA (9): Short-term exponential moving average of price
- SlowEMA (26): Long-term exponential moving average of price
- FastestEMA (5): Very short-term EMA for faster trend detection
- Aroon Up / Down (25): Measures trend strength and direction using recent highs/lows

Formulas (conceptual):
RSI        = 100 - (100 / (1 + RS))
EMA        = weighted moving average with more weight on recent prices
Volume SMA = mean(volume over window)
Aroon Up   = 100 * (periods since highest high / window)
Aroon Down = 100 * (periods since lowest low / window)

These indicators together help identify trend direction, momentum strength,
volume confirmation, and potential trend reversals.
"""

def calculate_indicators(df):
    df['RSI'] = RSIIndicator(df['close'], window=14).rsi()
    df['FastVolumeSMA'] = df['volume'].rolling(window=10).mean()
    df['SlowVolumeSMA'] = df['volume'].rolling(window=50).mean()
    df['FastEMA'] = df['close'].ewm(span=9, adjust=False).mean()
    df['SlowEMA'] = df['close'].ewm(span=26, adjust=False).mean()
    df['FastestEMA'] = df['close'].ewm(span=5, adjust=False).mean()
    aroon = ta.trend.AroonIndicator(df['high'], df['low'], window=25)
    df['AroonUp'] = aroon.aroon_up()
    df['AroonDown'] = aroon.aroon_down()
    return df

In [9]:
"""
Weighted Long and Short Signal Detection

These functions generate LONG and SHORT trading signals using a
multi-indicator weighted scoring framework. Instead of relying on a
single indicator, multiple technical, volatility, and volume signals
are combined to produce a more robust decision.

Core Concepts Used:
- Trend confirmation (Heikin-Ashi, EMAs, Aroon)
- Momentum shifts (MACD, RSI)
- Support/Resistance (Fibonacci 61.8%)
- Volatility regime detection (ATR + Hawkes process)
- Volume confirmation (volume vs average volume)

Key Steps:
1. Heikin-Ashi candles are computed to identify clean trend direction.
2. Fibonacci retracement levels are calculated to detect price proximity
   to the 61.8% level (key support/resistance zone).
3. MACD crossovers identify bullish or bearish momentum shifts.
4. ATR is used to measure volatility, and normalized ATR is passed
   through a Hawkes process to capture volatility clustering.
5. Volume is compared against its rolling average to confirm participation.
6. Additional indicators (RSI, EMAs, Aroon) refine trend strength and momentum.
7. Each condition contributes a predefined weight (score).
8. All scores are summed into a Total_Score.
9. A signal is triggered if Total_Score >= user-defined threshold.

Scoring Logic (Conceptual):
Total_Score = sum(weight_i * condition_i)

Where:
- condition_i is True or False
- weight_i reflects indicator importance

Hawkes Volatility Logic:
- Volatility input: normalized ATR
- Hawkes process emphasizes recent volatility bursts
- Signal strengthens when Hawkes volatility > current ATR

Output:
- Long_Signal: True when bullish weighted conditions dominate
- Short_Signal: True when bearish weighted conditions dominate

This weighted approach reduces false signals by requiring confirmation
across trend, momentum, volatility, and volume dimensions.
"""

def detect_weighted_long_signals(df,threshold,kappa):
    ha_df = calculate_heikin_ashi(df)
    df = pd.concat([df, ha_df], axis=1)
    fib_levels = calculate_fibonacci(df)
    df['Near_61.8%'] = np.abs(df['close'] - fib_levels["61.8%"]) <= (fib_levels["61.8%"] * 0.01)
    macd = MACD(df['close'])
    df['MACD'] = macd.macd()
    df['Signal'] = macd.macd_signal()
    df['Bullish_MACD'] = (df['MACD'] > df['Signal']) & (df['MACD'].shift(1) <= df['Signal'].shift(1))
    atr = AverageTrueRange(df['high'], df['low'], df['close'], window=14)
    df['ATR'] = atr.average_true_range()
    df['ATR_Check'] = df['ATR'] > (df['close'] * 0.015)
    df['norm_volatility'] = (df['ATR'] - df['ATR'].mean()) / df['ATR'].std()
    df['Hawkes_Volatility'] = hawkes_process(df['norm_volatility'], kappa)
    df['Avg_Volume'] = df['volume'].rolling(window=20).mean()
    df['Volume_Check'] = df['volume'] > (df['Avg_Volume'] * 1.5)
    df = calculate_indicators(df)
    df['Score_Heikin_Ashi'] = np.where(df['HA_Open'] == df['HA_Low'], 0.15, 0)
    df['Score_MACD'] = np.where(df['Bullish_MACD'], 0.4, 0)
    df['Score_Fibonacci'] = np.where(df['Near_61.8%'], 0.25, 0)
    df['Score_ATR'] = np.where(df['ATR_Check'], 0.1, 0)
    df['Score_Volume'] = np.where(df['Volume_Check'], 0.1, 0)
    df['Score_RSI'] = np.where((df['RSI'] > 30) & (df['RSI'] < 70), 0.2, 0)
    df['Score_EMA_Crossover'] = np.where(df['FastEMA'] > df['SlowEMA'], 0.3, 0)
    df['Score_Aroon'] = np.where(df['AroonUp'] > df['AroonDown'], 0.2, 0)
    df['Score_Hawkes_Volatility'] = np.where(df['Hawkes_Volatility'] > df['ATR'], 0.25, 0)
    df['Total_Score'] = (df['Score_Heikin_Ashi'] + df['Score_MACD'] + df['Score_Fibonacci'] +
                         df['Score_ATR'] + df['Score_Volume'] + df['Score_RSI'] +
                         df['Score_EMA_Crossover'] + df['Score_Aroon']+df['Score_Hawkes_Volatility'])
    df['Long_Signal'] = df['Total_Score'] >= threshold
    return df

def detect_weighted_short_signals(df, threshold,kappa):
    ha_df = calculate_heikin_ashi(df)
    df = pd.concat([df, ha_df], axis=1)
    fib_levels = calculate_fibonacci(df)
    df['Near_61.8%'] = np.abs(df['close'] - fib_levels["61.8%"]) <= (fib_levels["61.8%"] * 0.01)
    macd = MACD(df['close'])
    df['MACD'] = macd.macd()
    df['Signal'] = macd.macd_signal()
    df['Bearish_MACD'] = (df['MACD'] < df['Signal']) & (df['MACD'].shift(1) >= df['Signal'].shift(1))
    atr = AverageTrueRange(df['high'], df['low'], df['close'], window=14)
    df['ATR'] = atr.average_true_range()
    df['ATR_Check'] = df['ATR'] > (df['close'] * 0.015)
    df['norm_volatility'] = (df['ATR'] - df['ATR'].mean()) / df['ATR'].std()
    df['Hawkes_Volatility'] = hawkes_process(df['norm_volatility'], kappa)
    df['Avg_Volume'] = df['volume'].rolling(window=20).mean()
    df['Volume_Check'] = df['volume'] > (df['Avg_Volume'] * 1.5)
    df = calculate_indicators(df)
    df['Bearish_EMAs'] = (df['FastEMA'] < df['SlowEMA']) & (df['FastestEMA'] < df['FastEMA'])
    df['VWAP_Crossover'] = df['FastestEMA'] < df['FastEMA']
    df['VWAP_Crossover'] = (df['VWAP_Crossover'].shift(1) == False) & (df['VWAP_Crossover'] == True)
    df['RSI_Smoothed'] = df['RSI'].rolling(window=14).mean()
    df['RSI_Low'] = df['RSI_Smoothed'] < 30
    df['Weakening_Downward_Momentum'] = df['AroonDown'] < df['AroonUp']
    df['Score_EMAs'] = np.where(df['Bearish_EMAs'], 0.2, 0)
    df['Score_VWAP_Crossover'] = np.where(df['VWAP_Crossover'], 0.2, 0)
    df['Score_RSI'] = np.where(df['RSI_Low'], 0.2, 0)
    df['Score_Aroon'] = np.where(df['Weakening_Downward_Momentum'], 0.2, 0)
    df['Score_ATR'] = np.where(df['ATR_Check'], 0.1, 0)
    df['Score_Volume'] = np.where(df['Volume_Check'], 0.1, 0)
    df['Score_Hawkes_Volatility'] = np.where(df['Hawkes_Volatility'] > df['ATR'], 0.2, 0)
    df['Total_Score'] = (df['Score_EMAs'] + df['Score_VWAP_Crossover'] + df['Score_RSI'] +
                         df['Score_Aroon'] + df['Score_ATR'] + df['Score_Volume'] + df['Score_Hawkes_Volatility'])
    df['Short_Signal'] = df['Total_Score'] >= threshold
    return df

In [10]:
"""
Signal Aggregation and Trade Classification

This block combines the weighted LONG and SHORT signals into a single
final trading signal and assigns a trade type for each time step.

Steps:
1. Generate weighted long signals using detect_weighted_long_signals()
   with a specified threshold and Hawkes decay parameter (kappa).
2. Generate weighted short signals using detect_weighted_short_signals()
   with its own threshold and the same kappa.

Signal Encoding:
- Long_Signal: True  ->  1
- Short_Signal: True -> -1
- No signal:          0

3. Convert boolean long signals into numeric values:
   True  -> 1
   False -> 0

4. Convert boolean short signals into numeric values:
   True  -> -1
   False -> 0

5. Combine long and short signals into a single signal:
   signal[i] = long[i] + short[i]

Interpretation:
- signal =  1  -> Long bias
- signal = -1  -> Short bias
- signal =  0  -> Neutral / no trade

6. Map numeric signals to human-readable trade types:
-  1 -> "Long open"
-  0 -> "neutral"
- -1 -> "short open"

Purpose:
This logic ensures that long and short signals are mutually exclusive
and produces a clean, unified trading decision at each timestep.
"""



d1=detect_weighted_long_signals(data1, threshold=0.8,kappa=0.05)
d2=detect_weighted_short_signals(data1, threshold=0.6,kappa=0.05)
long=list(d1['Long_Signal'].to_numpy())
for i in range(1,len(long)):
    if long[i]==True:
        long[i]=1
    else:
        long[i]=0
short=list(d2['Short_Signal'].to_numpy())
for i in range(len(short)):
    if short[i]==True:
        short[i]=-1
    else:
        short[i]=0
signal=[0]*len(long)
for i in range(len(long)):
    signal[i]=short[i]+long[i]
trade_type=[]
for i in range(len(long)):
    if signal[i]==1:
        trade_type.append('Long open')
    elif signal[i]==0:
        trade_type.append('neutral')
    else:
        trade_type.append('short open')

In [ ]:
dat['signals']=signal
dat['trade_type']=trade_type
dat.drop()

,timestamp,open,high,low,close,volume,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume,signals,trade_type
0,2023-08-01 13:19:00,28902.48,28902.49,28902.48,28902.49,4.686580,1.354538e+05,258,0.893910,25836.224836,0,neutral
1,2023-08-01 13:18:00,28902.48,28902.49,28902.48,28902.49,4.775890,1.380351e+05,317,2.245460,64899.385195,0,neutral
2,2023-08-01 13:17:00,28908.52,28908.53,28902.48,28902.49,11.522630,3.330532e+05,451,2.708730,78290.170121,0,neutral
3,2023-08-01 13:16:00,28907.41,28912.74,28907.41,28908.53,15.896100,4.595556e+05,483,10.229810,295738.166916,0,neutral
4,2023-08-01 13:15:00,28896.00,28907.42,28893.03,28907.41,37.746570,1.090761e+06,686,16.504520,476955.246611,0,neutral
...,...,...,...,...,...,...,...,...,...,...,...,...
3125995,2017-08-17 04:04:00,4261.48,4261.48,4261.48,4261.48,0.140796,5.999993e+02,1,0.140796,599.999338,0,neutral
3125996,2017-08-17 04:03:00,4261.48,4261.48,4261.48,4261.48,0.012008,5.117185e+01,3,0.012008,51.171852,0,neutral
3125997,2017-08-17 04:02:00,4280.56,4280.56,4280.56,4280.56,0.261074,1.117543e+03,2,0.261074,1117.542921,0,neutral
3125998,2017-08-17 04:01:00,4261.48,4261.48,4261.48,4261.48,0.000000,0.000000e+00,0,0.000000,0.000000,0,neutral


In [12]:
dat.to_csv('BTC_Output.csv',index=False)
